In [1]:
import pandas as pd
import os
import numpy as np

In [2]:
os.chdir('/Users/be2297/Documents/Columbia/Research/Fulfillment/FulfillmentOptimization')
print(os.getcwd())

/Users/be2297/Documents/Columbia/Research/Fulfillment/FulfillmentOptimization


In [3]:
data = pd.read_csv('whole_experiment_n_samples_1000_resolve_samples_80.csv')

data['id_col'] = list(zip(data['graph_name'], data['demand_model'], data['weight_mode'], data['total_inventory']))
only_relaxation_data = data[data['inventory_placement'] == 'offline_relaxation']
relaxation_dict = dict(zip(only_relaxation_data['id_col'], only_relaxation_data['reward']))
data['relaxation_value'] = data['id_col'].map(relaxation_dict)
final_df = data[data['inventory_placement']!= 'offline_relaxation']
final_df['competitive_ratio'] = final_df['reward']/final_df['relaxation_value']
final_df

/var/folders/yj/y3ffb7_s6x33r52w0pq6wc780000gr/T/ipykernel_8084/2419662738.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_df['competitive_ratio'] = final_df['reward']/final_df['relaxation_value']


,graph_name,demand_model,weight_mode,total_inventory,inventory_placement,fulfillment_algorithm,reward,id_col,relaxation_value,competitive_ratio
1,complete,deter,uniform,30,offline_lp_rounding,offline,27.327497,"(complete, deter, uniform, 30)",27.327497,1.000000
2,complete,deter,uniform,30,offline_lp_rounding,myopic,24.932173,"(complete, deter, uniform, 30)",27.327497,0.912347
3,complete,deter,uniform,30,offline_lp_rounding,fluid_with_rejections,23.461150,"(complete, deter, uniform, 30)",27.327497,0.858518
4,complete,deter,uniform,30,offline_lp_rounding,offline_with_rejections,24.458393,"(complete, deter, uniform, 30)",27.327497,0.895010
5,complete,deter,uniform,30,offline_lp_rounding,fluid_with_rejections_resolving,26.173843,"(complete, deter, uniform, 30)",27.327497,0.957784
...,...,...,...,...,...,...,...,...,...,...
2245,starlike,correl,rewardbased,90,myopic_greedy,myopic,44.575504,"(starlike, correl, rewardbased, 90)",44.935815,0.991982
2246,starlike,correl,rewardbased,90,myopic_greedy,fluid_with_rejections,44.575504,"(starlike, correl, rewardbased, 90)",44.935815,0.991982
2247,starlike,correl,rewardbased,90,myopic_greedy,offline_with_rejections,44.708736,"(starlike, correl, rewardbased, 90)",44.935815,0.994947
2248,starlike,correl,rewardbased,90,myopic_greedy,fluid_with_rejections_resolving,44.559021,"(starlike, correl, rewardbased, 90)",44.935815,0.991615


In [10]:
def filter_weight_mode_and_fulfillment(df, weight_mode, fulfillment_algorithm):
    filtered_df = df[(df['fulfillment_algorithm'] == fulfillment_algorithm ) & (df['weight_mode'] == weight_mode) ]
    return filtered_df
    
filtered_df = filter_weight_mode_and_fulfillment(final_df, 'uniform', 'offline_with_rejections_resolving')

In [13]:
def create_sub_table(df, graph_name, demand_model):
    sub_df = df[(df['graph_name'] == graph_name) & (df['demand_model'] == demand_model)]
    pivoted_df = pd.pivot_table(sub_df, values = 'competitive_ratio', index = ['total_inventory'] ,columns = ['inventory_placement'] )
    return pivoted_df.reset_index()

create_sub_table(filtered_df, 'complete', 'deter')

inventory_placement,total_inventory,fluid_lp_rounding,fluid_lp_rounding_withscaling,myopic_greedy,offline_lp_rounding
0,30,0.957106,0.955023,0.954164,0.958363
1,45,0.968648,0.966051,0.966548,0.967150
2,60,0.989869,0.989896,0.990653,0.989910
3,75,0.988910,0.998615,0.999011,0.998997
4,90,0.987998,0.999667,0.999882,0.999869


In [14]:
def save_subtables_for_latex(df, graph_name_range, demand_model_range, fulfillment_algorithm, weight_mode):
    filtered_df = filter_weight_mode_and_fulfillment(df, weight_mode, fulfillment_algorithm)
    for graph_name in graph_name_range:
        for demand_model in demand_model_range:
            sub_df = create_sub_table(filtered_df, graph_name, demand_model)
            sub_df.to_csv(f'{weight_mode}_{graph_name}_{demand_model}.csv')

In [17]:
graph_name_range = ['complete', 'longchain', 'starlike']
demand_model_range = ['deter', 'indep', 'correl']
fulfillment_algorithm = 'offline_with_rejections_resolving'
weight_mode = 'uniform'

save_subtables_for_latex(final_df, graph_name_range, demand_model_range, fulfillment_algorithm, weight_mode)

In [18]:
final_df.head(
)

,graph_name,demand_model,weight_mode,total_inventory,inventory_placement,fulfillment_algorithm,reward,id_col,relaxation_value,competitive_ratio
1,complete,deter,uniform,30,offline_lp_rounding,offline,27.327497,"(complete, deter, uniform, 30)",27.327497,1.000000
2,complete,deter,uniform,30,offline_lp_rounding,myopic,24.932173,"(complete, deter, uniform, 30)",27.327497,0.912347
3,complete,deter,uniform,30,offline_lp_rounding,fluid_with_rejections,23.461150,"(complete, deter, uniform, 30)",27.327497,0.858518
4,complete,deter,uniform,30,offline_lp_rounding,offline_with_rejections,24.458393,"(complete, deter, uniform, 30)",27.327497,0.895010
5,complete,deter,uniform,30,offline_lp_rounding,fluid_with_rejections_resolving,26.173843,"(complete, deter, uniform, 30)",27.327497,0.957784


In [4]:
final_df = final_df[final_df['fulfillment_algorithm']!= 'offline']
best_cr = final_df.groupby(['graph_name', 'demand_model', 'weight_mode' ,'inventory_placement','total_inventory'])['competitive_ratio'].apply(max)
best_cr
best_cr_dict = best_cr.to_dict()
final_df['aux_tuple'] = list(zip(final_df['graph_name'], final_df['demand_model'], final_df['weight_mode'], final_df['inventory_placement'],final_df['total_inventory']))
final_df['best_cr'] = final_df['aux_tuple'].map(best_cr_dict)
final_df.head(10)

/var/folders/yj/y3ffb7_s6x33r52w0pq6wc780000gr/T/ipykernel_8084/2473350021.py:2: FutureWarning: The provided callable <built-in function max> is currently using np.maximum.reduce. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string np.maximum.reduce instead.
  best_cr = final_df.groupby(['graph_name', 'demand_model', 'weight_mode' ,'inventory_placement','total_inventory'])['competitive_ratio'].apply(max)
/var/folders/yj/y3ffb7_s6x33r52w0pq6wc780000gr/T/ipykernel_8084/2473350021.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_df['aux_tuple'] = list(zip(final_df['graph_name'], final_df['demand_model'], final_df['weight_mode'], final_df['inventory_placement'],final_df['total_

,graph_name,demand_model,weight_mode,total_inventory,inventory_placement,fulfillment_algorithm,reward,id_col,relaxation_value,competitive_ratio,aux_tuple,best_cr
2,complete,deter,uniform,30,offline_lp_rounding,myopic,24.932173,"(complete, deter, uniform, 30)",27.327497,0.912347,"(complete, deter, uniform, offline_lp_rounding...",0.958363
3,complete,deter,uniform,30,offline_lp_rounding,fluid_with_rejections,23.461150,"(complete, deter, uniform, 30)",27.327497,0.858518,"(complete, deter, uniform, offline_lp_rounding...",0.958363
4,complete,deter,uniform,30,offline_lp_rounding,offline_with_rejections,24.458393,"(complete, deter, uniform, 30)",27.327497,0.895010,"(complete, deter, uniform, offline_lp_rounding...",0.958363
5,complete,deter,uniform,30,offline_lp_rounding,fluid_with_rejections_resolving,26.173843,"(complete, deter, uniform, 30)",27.327497,0.957784,"(complete, deter, uniform, offline_lp_rounding...",0.958363
6,complete,deter,uniform,30,offline_lp_rounding,offline_with_rejections_resolving,26.189657,"(complete, deter, uniform, 30)",27.327497,0.958363,"(complete, deter, uniform, offline_lp_rounding...",0.958363
8,complete,deter,uniform,30,fluid_lp_rounding,myopic,24.898457,"(complete, deter, uniform, 30)",27.327497,0.911114,"(complete, deter, uniform, fluid_lp_rounding, 30)",0.957106
9,complete,deter,uniform,30,fluid_lp_rounding,fluid_with_rejections,23.079916,"(complete, deter, uniform, 30)",27.327497,0.844567,"(complete, deter, uniform, fluid_lp_rounding, 30)",0.957106
10,complete,deter,uniform,30,fluid_lp_rounding,offline_with_rejections,24.414722,"(complete, deter, uniform, 30)",27.327497,0.893412,"(complete, deter, uniform, fluid_lp_rounding, 30)",0.957106
11,complete,deter,uniform,30,fluid_lp_rounding,fluid_with_rejections_resolving,26.117066,"(complete, deter, uniform, 30)",27.327497,0.955706,"(complete, deter, uniform, fluid_lp_rounding, 30)",0.957106
12,complete,deter,uniform,30,fluid_lp_rounding,offline_with_rejections_resolving,26.155313,"(complete, deter, uniform, 30)",27.327497,0.957106,"(complete, deter, uniform, fluid_lp_rounding, 30)",0.957106


In [17]:
final_df['is_best'] = np.where(final_df['competitive_ratio'] == final_df['best_cr'],1,0)
final_df.head(10)

/var/folders/yj/y3ffb7_s6x33r52w0pq6wc780000gr/T/ipykernel_8084/19009953.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_df['is_best'] = np.where(final_df['competitive_ratio'] == final_df['best_cr'],1,0)


,graph_name,demand_model,weight_mode,total_inventory,inventory_placement,fulfillment_algorithm,reward,id_col,relaxation_value,competitive_ratio,aux_tuple,best_cr,is_best,within_05_perc
2,complete,deter,uniform,30,offline_lp_rounding,myopic,24.932173,"(complete, deter, uniform, 30)",27.327497,0.912347,"(complete, deter, uniform, offline_lp_rounding...",0.958363,0,0
3,complete,deter,uniform,30,offline_lp_rounding,fluid_with_rejections,23.461150,"(complete, deter, uniform, 30)",27.327497,0.858518,"(complete, deter, uniform, offline_lp_rounding...",0.958363,0,0
4,complete,deter,uniform,30,offline_lp_rounding,offline_with_rejections,24.458393,"(complete, deter, uniform, 30)",27.327497,0.895010,"(complete, deter, uniform, offline_lp_rounding...",0.958363,0,0
5,complete,deter,uniform,30,offline_lp_rounding,fluid_with_rejections_resolving,26.173843,"(complete, deter, uniform, 30)",27.327497,0.957784,"(complete, deter, uniform, offline_lp_rounding...",0.958363,0,1
6,complete,deter,uniform,30,offline_lp_rounding,offline_with_rejections_resolving,26.189657,"(complete, deter, uniform, 30)",27.327497,0.958363,"(complete, deter, uniform, offline_lp_rounding...",0.958363,1,1
8,complete,deter,uniform,30,fluid_lp_rounding,myopic,24.898457,"(complete, deter, uniform, 30)",27.327497,0.911114,"(complete, deter, uniform, fluid_lp_rounding, 30)",0.957106,0,0
9,complete,deter,uniform,30,fluid_lp_rounding,fluid_with_rejections,23.079916,"(complete, deter, uniform, 30)",27.327497,0.844567,"(complete, deter, uniform, fluid_lp_rounding, 30)",0.957106,0,0
10,complete,deter,uniform,30,fluid_lp_rounding,offline_with_rejections,24.414722,"(complete, deter, uniform, 30)",27.327497,0.893412,"(complete, deter, uniform, fluid_lp_rounding, 30)",0.957106,0,0
11,complete,deter,uniform,30,fluid_lp_rounding,fluid_with_rejections_resolving,26.117066,"(complete, deter, uniform, 30)",27.327497,0.955706,"(complete, deter, uniform, fluid_lp_rounding, 30)",0.957106,0,1
12,complete,deter,uniform,30,fluid_lp_rounding,offline_with_rejections_resolving,26.155313,"(complete, deter, uniform, 30)",27.327497,0.957106,"(complete, deter, uniform, fluid_lp_rounding, 30)",0.957106,1,1


In [21]:
final_df[final_df['fulfillment_algorithm'] == 'offline_with_rejections_resolving']['is_best'].sum()

310

In [6]:
pivot_table = pd.pivot_table(data = final_df, index = 'fulfillment_algorithm', columns = 'inventory_placement', values = 'is_best')*100
pivot_table

inventory_placement,fluid_lp_rounding,fluid_lp_rounding_withscaling,myopic_greedy,offline_lp_rounding
fulfillment_algorithm,,,,
fluid_with_rejections,13.333333,4.444444,10.000000,5.555556
fluid_with_rejections_resolving,15.555556,4.444444,5.555556,2.222222
myopic,17.777778,4.444444,8.888889,6.666667
offline_with_rejections,21.111111,8.888889,17.777778,10.000000
offline_with_rejections_resolving,90.000000,86.666667,80.000000,87.777778


In [7]:
final_df['within_05_perc'] = np.where( final_df['competitive_ratio'] > 0.995*final_df['best_cr'],1,0)
final_df

/var/folders/yj/y3ffb7_s6x33r52w0pq6wc780000gr/T/ipykernel_8084/1659638229.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_df['within_05_perc'] = np.where( final_df['competitive_ratio'] > 0.995*final_df['best_cr'],1,0)


,graph_name,demand_model,weight_mode,total_inventory,inventory_placement,fulfillment_algorithm,reward,id_col,relaxation_value,competitive_ratio,aux_tuple,best_cr,is_best,within_05_perc
2,complete,deter,uniform,30,offline_lp_rounding,myopic,24.932173,"(complete, deter, uniform, 30)",27.327497,0.912347,"(complete, deter, uniform, offline_lp_rounding...",0.958363,0,0
3,complete,deter,uniform,30,offline_lp_rounding,fluid_with_rejections,23.461150,"(complete, deter, uniform, 30)",27.327497,0.858518,"(complete, deter, uniform, offline_lp_rounding...",0.958363,0,0
4,complete,deter,uniform,30,offline_lp_rounding,offline_with_rejections,24.458393,"(complete, deter, uniform, 30)",27.327497,0.895010,"(complete, deter, uniform, offline_lp_rounding...",0.958363,0,0
5,complete,deter,uniform,30,offline_lp_rounding,fluid_with_rejections_resolving,26.173843,"(complete, deter, uniform, 30)",27.327497,0.957784,"(complete, deter, uniform, offline_lp_rounding...",0.958363,0,1
6,complete,deter,uniform,30,offline_lp_rounding,offline_with_rejections_resolving,26.189657,"(complete, deter, uniform, 30)",27.327497,0.958363,"(complete, deter, uniform, offline_lp_rounding...",0.958363,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2245,starlike,correl,rewardbased,90,myopic_greedy,myopic,44.575504,"(starlike, correl, rewardbased, 90)",44.935815,0.991982,"(starlike, correl, rewardbased, myopic_greedy,...",0.995928,0,1
2246,starlike,correl,rewardbased,90,myopic_greedy,fluid_with_rejections,44.575504,"(starlike, correl, rewardbased, 90)",44.935815,0.991982,"(starlike, correl, rewardbased, myopic_greedy,...",0.995928,0,1
2247,starlike,correl,rewardbased,90,myopic_greedy,offline_with_rejections,44.708736,"(starlike, correl, rewardbased, 90)",44.935815,0.994947,"(starlike, correl, rewardbased, myopic_greedy,...",0.995928,0,1
2248,starlike,correl,rewardbased,90,myopic_greedy,fluid_with_rejections_resolving,44.559021,"(starlike, correl, rewardbased, 90)",44.935815,0.991615,"(starlike, correl, rewardbased, myopic_greedy,...",0.995928,0,1


In [110]:
pivot_table2 = pd.pivot_table(data = final_df, index = 'fulfillment_algorithm', columns = 'inventory_placement', values = 'within_1_perc')*100
pivot_table2

inventory_placement,fluid_lp_rounding,fluid_lp_rounding_withscaling,myopic_greedy,offline_lp_rounding
fulfillment_algorithm,,,,
fluid_with_rejections,15.555556,26.666667,32.222222,20.000000
fluid_with_rejections_resolving,20.000000,27.777778,34.444444,22.222222
myopic,35.555556,22.222222,27.777778,17.777778
offline_with_rejections,52.222222,68.888889,58.888889,45.555556
offline_with_rejections_resolving,100.000000,100.000000,100.000000,100.000000


In [13]:
grouped_df = final_df.groupby(['fulfillment_algorithm', 'inventory_placement'])[['competitive_ratio']].apply('mean').unstack()*100
grouped_df.columns = grouped_df.columns.swaplevel(0, 1)
grouped_df = grouped_df.sort_index(axis=1)



# Define new column order (keeping MultiIndex structure)
column_order = [
    ("offline_lp_rounding", "competitive_ratio"),
    ("fluid_lp_rounding", "competitive_ratio"),
    ("fluid_lp_rounding_withscaling", "competitive_ratio"),
    ("myopic_greedy", "competitive_ratio")
]

# Define new row order
row_order = [
    "myopic", 
    "fluid_with_rejections", 
    "offline_with_rejections", 
    "fluid_with_rejections_resolving", 
    "offline_with_rejections_resolving"
]

# Reorder columns and rows
grouped_df = grouped_df.reindex(index=row_order, columns=column_order)

# Display reordered table
grouped_df


inventory_placement,offline_lp_rounding,fluid_lp_rounding,fluid_lp_rounding_withscaling,myopic_greedy
,competitive_ratio,competitive_ratio,competitive_ratio,competitive_ratio
fulfillment_algorithm,,,,
myopic,95.646582,93.335458,96.009938,96.567985
fluid_with_rejections,93.599923,87.031635,93.863144,94.326285
offline_with_rejections,97.416637,94.385073,96.544186,96.938828
fluid_with_rejections_resolving,95.666785,91.182360,95.476272,95.961071
offline_with_rejections_resolving,98.566194,95.381366,97.369093,97.829641


In [16]:
print((grouped_df/100).to_latex(multicolumn=True, multirow=True, float_format="%.3f"))

\begin{tabular}{lrrrr}
\toprule
inventory_placement & offline_lp_rounding & fluid_lp_rounding & fluid_lp_rounding_withscaling & myopic_greedy \\
 & competitive_ratio & competitive_ratio & competitive_ratio & competitive_ratio \\
fulfillment_algorithm &  &  &  &  \\
\midrule
myopic & 0.956 & 0.933 & 0.960 & 0.966 \\
fluid_with_rejections & 0.936 & 0.870 & 0.939 & 0.943 \\
offline_with_rejections & 0.974 & 0.944 & 0.965 & 0.969 \\
fluid_with_rejections_resolving & 0.957 & 0.912 & 0.955 & 0.960 \\
offline_with_rejections_resolving & 0.986 & 0.954 & 0.974 & 0.978 \\
\bottomrule
\end{tabular}



In [101]:
loaded_df = pd.read_csv('fulfillment_table.csv', header=[0, 1], index_col=0)
print(loaded_df.to_latex(multicolumn=True, multirow=True, float_format="%.2f"))

\begin{tabular}{lrrrrrrrr}
\toprule
inventory_placement & \multicolumn{2}{r}{fluid_lp_rounding} & \multicolumn{2}{r}{fluid_lp_rounding_withscaling} & \multicolumn{2}{r}{myopic_greedy} & \multicolumn{2}{r}{offline_lp_rounding} \\
 & is_best & within_1_perc & is_best & within_1_perc & is_best & within_1_perc & is_best & within_1_perc \\
fulfillment_algorithm &  &  &  &  &  &  &  &  \\
\midrule
fluid_with_rejections & 13.33 & 15.56 & 4.44 & 32.22 & 10.00 & 37.78 & 5.56 & 20.00 \\
fluid_with_rejections_resolving & 15.56 & 26.67 & 4.44 & 33.33 & 5.56 & 38.89 & 2.22 & 27.78 \\
myopic & 17.78 & 38.89 & 4.44 & 27.78 & 8.89 & 34.44 & 6.67 & 21.11 \\
offline_with_rejections & 21.11 & 56.67 & 8.89 & 73.33 & 17.78 & 66.67 & 10.00 & 54.44 \\
offline_with_rejections_resolving & 90.00 & 100.00 & 86.67 & 100.00 & 80.00 & 100.00 & 87.78 & 100.00 \\
\bottomrule
\end{tabular}



In [81]:
grouped_df.columns

MultiIndex([(            'fluid_lp_rounding',       'is_best'),
            (            'fluid_lp_rounding', 'within_1_perc'),
            ('fluid_lp_rounding_withscaling',       'is_best'),
            ('fluid_lp_rounding_withscaling', 'within_1_perc'),
            (                'myopic_greedy',       'is_best'),
            (                'myopic_greedy', 'within_1_perc'),
            (          'offline_lp_rounding',       'is_best'),
            (          'offline_lp_rounding', 'within_1_perc')],
           names=['inventory_placement', None])

In [90]:
grouped_2 = final_df.groupby(['graph_name', 'demand_model', 'weight_mode', 'total_inventory','inventory_placement'])['is_best'].apply('sum')
len(grouped_2[grouped_2==1])/len(grouped_2)

0.9055555555555556

In [93]:
np.mean(grouped_2[grouped_2>1])

3.676470588235294

In [102]:
# Define new column order (keeping MultiIndex structure)
column_order = [
    ("offline_lp_rounding", "is_best"), ("offline_lp_rounding", "within_1_perc"),
    ("fluid_lp_rounding", "is_best"), ("fluid_lp_rounding", "within_1_perc"),
    ("fluid_lp_rounding_withscaling", "is_best"), ("fluid_lp_rounding_withscaling", "within_1_perc"),
    ("myopic_greedy", "is_best"), ("myopic_greedy", "within_1_perc")
]

# Define new row order
row_order = [
    "myopic", 
    "fluid_with_rejections", 
    "offline_with_rejections", 
    "fluid_with_rejections_resolving", 
    "offline_with_rejections_resolving"
]

# Reorder columns and rows
loaded_df = loaded_df.reindex(index=row_order, columns=column_order)

# Display reordered table
loaded_df


inventory_placement               offline_lp_rounding                \
                                              is_best within_1_perc   
fulfillment_algorithm                                                 
myopic                                       6.666667     21.111111   
fluid_with_rejections                        5.555556     20.000000   
offline_with_rejections                     10.000000     54.444444   
fluid_with_rejections_resolving              2.222222     27.777778   
offline_with_rejections_resolving           87.777778    100.000000   

inventory_placement               fluid_lp_rounding                \
                                            is_best within_1_perc   
fulfillment_algorithm                                               
myopic                                    17.777778     38.888889   
fluid_with_rejections                     13.333333     15.555556   
offline_with_rejections                   21.111111     56.666667   
fluid_with_rejections_resolving           15.555556     26.666667   
offline_with_rejections_resolving         90.000000    100.000000   

inventory_placement               fluid_lp_rounding_withscaling                \
                                                        is_best within_1_perc   
fulfillment_algorithm                                                           
myopic                                                 4.444444     27.777778   
fluid_with_rejections                                  4.444444     32.222222   
offline_with_rejections                                8.888889     73.333333   
fluid_with_rejections_resolving                        4.444444     33.333333   
offline_with_rejections_resolving                     86.666667    100.000000   

inventory_placement               myopic_greedy                
                                        is_best within_1_perc  
fulfillment_algorithm                                          
myopic                                 8.888889     34.444444  
fluid_with_rejections                 10.000000     37.777778  
offline_with_rejections               17.777778     66.666667  
fluid_with_rejections_resolving        5.555556     38.888889  
offline_with_rejections_resolving     80.000000    100.000000

In [103]:
loaded_df.to_csv('fulfillment_policy_table.csv')

In [104]:
print(loaded_df.to_latex(multicolumn=True, multirow=True, float_format="%.2f"))

\begin{tabular}{lrrrrrrrr}
\toprule
inventory_placement & \multicolumn{2}{r}{offline_lp_rounding} & \multicolumn{2}{r}{fluid_lp_rounding} & \multicolumn{2}{r}{fluid_lp_rounding_withscaling} & \multicolumn{2}{r}{myopic_greedy} \\
 & is_best & within_1_perc & is_best & within_1_perc & is_best & within_1_perc & is_best & within_1_perc \\
fulfillment_algorithm &  &  &  &  &  &  &  &  \\
\midrule
myopic & 6.67 & 21.11 & 17.78 & 38.89 & 4.44 & 27.78 & 8.89 & 34.44 \\
fluid_with_rejections & 5.56 & 20.00 & 13.33 & 15.56 & 4.44 & 32.22 & 10.00 & 37.78 \\
offline_with_rejections & 10.00 & 54.44 & 21.11 & 56.67 & 8.89 & 73.33 & 17.78 & 66.67 \\
fluid_with_rejections_resolving & 2.22 & 27.78 & 15.56 & 26.67 & 4.44 & 33.33 & 5.56 & 38.89 \\
offline_with_rejections_resolving & 87.78 & 100.00 & 90.00 & 100.00 & 86.67 & 100.00 & 80.00 & 100.00 \\
\bottomrule
\end{tabular}

